# Université Paul Sabatier

M1IAFA - Recherche d'information

**TP 3**

Enseignants : Lynda Tamine et Jesús Lovón

Notebook proposé par : José G. Moreno


2024

---

💡 Penser à développer des scripts et fonctions auxiliaires qui vont permettre réutiliser les commandes récurrentes dans ce TP et les prochains. Ceci aussi vous permettra de garder de bon pratique du code et faciliter vos débogages.

---


### Attention ❗❗ Pour la note du TP :
🚨 *Questions de code* : Remplir le code manquant dans les parties correspondantes (le code commenté bénéficie des meilleures notes).

🚨 *Questions ouvertes* : Ecrivez votre réponse textuelle sous forme de commentaires dans les cellules correspondantes.

🚨 *Laissez vos sorties* pour les cellules où vous écrivez le code. Les sorties vides (notebook ou cellules non exécutées) correspondent à 0 points.

---

# TP 3. Évaluation d’un système de recherche d'information

## Introduction



L'évaluation est une étape complexe dans la recherche d'information. Une des conférences qui a largement aidé à l'avancement en cette matière est la conférence TREC (http://trec.nist.gov/).


**Dans ce TP nous nous intéressons à l'utilisation d'un de ces outils pour l'évaluation des moteurs des recherche.**


Pour l'évaluation nous avons besoin d'un fichier qui contient la « vérité de terrain » ou « gold standard » qui est normalement appelé **qrel**. Ce fichier contient pour chaque requête les identifiants des documents pertinents et non-pertinents. Également, il est nécessaire d'avoir des fichiers des résultats du moteur de recherche à évaluer.

Dans ce TP, nous allons utiliser un seul fichier qrel et plusieurs fichiers des résultats (chaque fichier des résultats sera évalué).

En continuation avec le TP2, considérez la phrase « Thomas and Mario are strikers playing in Munich ». Elle sera transformée en 3 requêtes  « Thomas », « Mario » et « Munich ». Chaque requête aura de documents considérés comme correctes (relevant) et incorrectes (no relevant).  La recherche de documents sera faite par votre système de recherche d’information. Cependant, le fait de dire qu’un document est relevant est une étape manuelle.


(A) Nous allons considérer les documents suivants comme **relevant** pour chaque requête :

> **Thomas** and **Mario** are strikers playing in **Munich**
>
>Thomas <br>
>* http://simple.wikipedia.org/wiki/Thomas_Müller
>
>
>Mario <br>
>* http://simple.wikipedia.org/wiki/Mario_Gómez <br>
>* http://simple.wikipedia.org/wiki/Mario_Götze
>
>
>Munich <br>
>* http://simple.wikipedia.org/wiki/FC_Bayern_Munich

Maintenant, il suffit d’utiliser vos résultats de chaque requête dans le format TREC pour les évaluer. Pour simplicité, nous allons utiliser la librairie [pytrec_eval](https://github.com/cvangysel/pytrec_eval) qui est un wrapper du logiciel [trec_eval](https://trec.nist.gov/trec_eval/)

Pour information, voici le fichier en format qrel pour les 3 requêtes précédentes de (A) :

```
101 0 Thomas_Müller 1
101 0 Thomas_Edison 0
101 0 Thomas_the_Apostle 0
102 0 Mario_Gómez 1
102 0 Mario_Götze 1
103 0 FC_Bayern_Munich 1
```

Notez que nous allons utiliser pytrec_eval, qui utilise un dictionnaire pour le qrel au lieu d'un fichier.

Notez que la première colonne est l’identifiant de la requête (nous avons trois valeurs différentes, une pour chaque requête), suivie de zéro (0), suivi de l’identifiant du document annoté (le titre de la page Wikipédia) et une valeur pour dire si le document est relevant (1) ou non (0). Notez aussi que les qrels contient des documents pertinents et des documents non-pertinents.

Puis il faut créer le fichier des résultats avec la sortie de votre programme fait pendant les TPs précédents. Pour information, voici un fichier résultat d’un système :

```
101	Q0	Thomas_Edison	1	  5.5	STANDARD
101	Q0	Thomas_Müller	2	  4.4	STANDARD
101	Q0	Thomas_the_Apostle	3	  3.3	STANDARD
101	Q0	Isiah_Thomas	4	  2.2	STANDARD
101	Q0	Thomas_Aquinas	5	  1.1	STANDARD
102	Q0	Mario	1	  5.5	STANDARD
102	Q0	Super_Mario	2	  4.4	STANDARD
102	Q0	Super_Mario_Bros.	3	  3.3	STANDARD
102	Q0	Super_Mario_Bros._2	4	  2.2	STANDARD
102	Q0	Mario_(series)	5	  1.1	STANDARD
102	Q0	Super_Mario_World	6	  1.0	STANDARD
102	Q0	Super_Mario_Bros._3	7	  0.9	STANDARD
102	Q0	New_Super_Mario_Bros.	8	  0.8	STANDARD
102	Q0	Mario_Gómez	9	  0.7	STANDARD
102	Q0	Mario_Party_4	10	  0.6	STANDARD
103	Q0	Munich	1	  5.5	STANDARD
103	Q0	FC_Bayern_Munich	2	  4.4	STANDARD
103	Q0	Munich_Airport	3	  3.3	STANDARD
103	Q0	Munich_Agreement	4	  2.2	STANDARD
103	Q0	Munich_Rural_District	5	  1.1	STANDARD
```

Notez, que comme pour les qrels, pytrec_eval utilise un dictionnaire pour le résultat d’un système au lieu d'un fichier.

La première colonne est l’identifiant de la requête (la même que pour le qrel), suivie de zéro (Q0), suivi de l’identifiant du document retrouvé par votre système (le titre de la page Wikipédia), suivi de la position du document dans les résultats, suivi de la valeur de similarité donnée par le modèle de poids choisi et de l’identifiant du système (votre nom par exemple).

Une fois construis les fichiers qrels et résultats, nous pouvons utiliser le logiciel d'évaluation trec_eval pour obtenir les résultats de l'évaluation. Cependant, pour simplicité nous allons utiliser pytrec_eval. Donc, pour pytrec_eval, il suffit de déclarer les deux dictionnaires (qrel et run) et en suite appeler la méthode ```relevanceEvaluator``` comme indiqué dans l'exemple ci-dessous.

## Exemple
Pour clarifier tout ce processus, nous allons considérer l'exemple suivant.



1. Tout d'abord, nous installons la bibliothèque pytrec_eval et importons des bibliothèques complémentaires pour pytrec_eval, pandas et la manipulation JSON.

In [42]:
!pip install pytrec_eval

In [43]:
import pytrec_eval
import json
import pandas as pd

2. Maintenant, nous allons créer le fichier **qrel** qui indique quels sont les titres pertinents et non pertinents de Wikipédia pour chaque requête suivant l'exemple donné précédemment.

De même, dans la variable **run**, nous allons créer un dictionnaire avec quelques résultats fictifs pour chaque requête dans le but de montrer comment l'évaluation est effectuée.

In [44]:
qrel = {
    '101': {
        'Thomas_Müller': 1,
        'Thomas_Edison': 0,
        'Thomas_the_Apostle': 0,
    },
    '102': {
        'Mario_Gómez': 1,
        'Mario_Götze': 1,
    },
    '103': {
        'FC_Bayern_Munich': 1,
    },
}


In [45]:
run = {
    '101': {
        'Thomas_Edison': 5.5,
        'Thomas_Müller': 4.4,
        'Thomas_the_Apostle': 3.3,
        'Isiah_Thomas': 2.2,
        'Thomas_Aquinas': 1.1,
    },
    '102': {
        'Mario': 10.10,
        'Super_Mario': 9.9,
        'Super_Mario_Bros.': 8.8,
        'Super_Mario_Bros._2': 7.7,
        'Mario_(series)': 6.6,
        'Super_Mario_World': 5.5,
        'Super_Mario_Bros._3': 4.4,
        'New_Super_Mario_Bros.': 3.3,
        'Mario_Gómez': 2.2,
        'Mario_Party_4': 1.1,
    },
    '103': {
        'Munich': 5.5,
        'FC_Bayern_Munich': 4.4,
        'Munich_Airport': 3.3,
        'Munich_Agreement': 2.2,
        'Munich_Rural_District': 1.1,
    },
}


3. Évaluation de l'exemple.

Enfin, avec les commandes suivantes, nous calculons les métriques (MAP et NDCG dans ce cas) en utilisant les variables **qrel** et **run** (consultez vos notes de cours pour plus de méthodes et de détails sur ces métriques).

In [46]:
evaluator = pytrec_eval.RelevanceEvaluator(
    qrel, {'map', 'ndcg'})

pd.DataFrame(evaluator.evaluate(run)).T

,map,ndcg
101,0.500000,0.630930
102,0.055556,0.184576
103,0.500000,0.630930


Chaque clé corresponde au résultat d’une métrique d’évaluation pour les trois requêtes.

#1. Requêtes



Création de la variable ```run```. La variable ```run``` précédente était fictive. Créez une variable "run" en utilisant l'index des TPs précedents.


Utilisez les suivants requêtes dans votre système et générez les résultats dans le format décrit précédemment (variable ```run```).


ATTTENTION !  X+i correspond à l'ID de la requête, où la variable i correspond à une entité. Par exemple, 101 pour Thomas (Thomas Müller), et 201 pour Leo (Lionel Messi).


```
ID:100+i
Thomas and Mario are strikers playing in Munich

ID:200+i
Leo scored two goals and assisted Puyol to ensure a 4–0 quarter-final victory over Bayern

ID:300+i
Skype software for Mac

ID:400+i
Cowboys fans petition Obama to oust Jones

ID:500+i
Kate and Henry are known for being devoted to the Anglican church
```



In [47]:
# VOTRE CODE ICI
# FAITES LES INSTALLATIONS ET LES IMPORTATIONS REQUIS COMME DANS LES TPS PRÉCÉDENTS
# déclaration de la variable JAVA_HOME
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
!export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

  Cloning https://github.com/terrier-org/pyterrier.git to /tmp/pip-install-1lhnogkq/python-terrier_85f1f160120b4c26b924a54b35818674
  Running command git clone --filter=blob:none --quiet https://github.com/terrier-org/pyterrier.git /tmp/pip-install-1lhnogkq/python-terrier_85f1f160120b4c26b924a54b35818674
  Resolved https://github.com/terrier-org/pyterrier.git to commit 81f20bda483f6044bda89f717ae863e62bab42e5
  Preparing metadata (setup.py) ... done


In [48]:
import pyterrier as pt
if not pt.started():
  pt.init()

In [49]:
!unzip -o pd_index.zip

Archive:  pd_index.zip
  inflating: pd_index/data.meta.zdata  
  inflating: pd_index/data.document.fsarrayfile  
  inflating: pd_index/data.meta-0.fsomapfile  
  inflating: pd_index/data.lexicon.fsomapid  
  inflating: pd_index/data.properties  
  inflating: pd_index/data.lexicon.fsomapfile  
  inflating: pd_index/data.direct.bf  
  inflating: pd_index/data.lexicon.fsomaphash  
  inflating: pd_index/data.meta.idx  
  inflating: pd_index/data.inverted.bf  


In [73]:
# Pour cree un index
#indexref2 = pt.IndexRef.of("/content/drive/MyDrive/pd_index/data.properties")
indexref2 = pt.IndexRef.of("/content/pd_index/data.properties")

In [53]:
def basic_term_search_dictionnary(query, method):
  termToSearch = [x for x in query.split() if x[0]==x[0].upper()]
  print(termToSearch)
  for term, page in zip(termToSearch,['Thomas_Müller','Mario Gtze', 'FC Bayern Munich']):
    dfres = pt.BatchRetrieve(indexref2, wmodel=method, metadata=["docno","title","url"]).search(term)[['title','score']]
    print(dfres.head(10))
    print("Position de la page attendue dans les résultats :\n",dfres[dfres['title']==page],"\n\n\n")


query = "Thomas and Mario are strikers playing in Munich"
basic_term_search_dictionnary(query, "BM25")

['Thomas', 'Mario', 'Munich']
                         title      score
0             Thomas (surname)  11.700841
1                 Saint Thomas  11.597729
2           Thomas the Apostle  11.447421
3                Thomas County  11.387148
4           Thomas Wedgwood IV  11.210921
5                 Sarah Thomas  11.152142
6                 Helen Thomas  11.145990
7             Gospel of Thomas  11.121265
8      Thomas White (merchant)  11.026214
9  Mallappally Marthoma Church  11.013052
Position de la page attendue dans les résultats :
 Empty DataFrame
Columns: [title, score]
Index: [] 



                          title      score
0                Princess Peach  16.478378
1        Mario (disambiguation)  16.451796
2  Super Mario (disambiguation)  16.419316
3                         Mario  16.395732
4              Charles Martinet  16.349001
5                    Bowser Jr.  16.214048
6                         Wario  16.177717
7                         Luigi  16.132403
8               

In [54]:
def basic_dictionary(query_with_id, method, nombre_resultats):

    # Séparer l'identifiant de la requête du reste de la chaîne
    query_id_str, query = query_with_id.split(": ", 1)
    query_id_base = int(query_id_str)

    # Extraction des termes de recherche qui commencent par une majuscule
    terms_to_search = [x for x in query.split() if x[0].isupper()]

    run = {}


    # Pour chaque terme recherché
    for i, term in enumerate(terms_to_search, start=1):
        # Générer l'identifiant pour ce terme (par exemple, 101, 102, etc.)
        term_id = str(query_id_base + i)

        # Effectuer la recherche pour le terme en utilisant pt.BatchRetrieve ou un système équivalent
        # Ici, dfres représenterait les résultats réels de la recherche pour 'term'
        dfres = pt.BatchRetrieve(indexref2, wmodel=method, metadata=["docno", "title", "url"]).search(term)[['title', 'score']]

        # Initialiser une entrée dans le dictionnaire pour ce terme
        run[term_id] = {}

        # Parcourir les résultats de recherche et les ajouter au dictionnaire
        for _, row in dfres.iterrows():
            # Utiliser le titre comme clé et le score comme valeur
            run[term_id][row['title']] = row['score']

    return run

In [55]:
# Exemple d'utilisation (l'exécution réelle nécessite l'environnement approprié avec pt.BatchRetrieve configuré)
query1 = "100: New research suggests coffee may reduce the risk of liver disease"
query2 = "200: SpaceX launches Falcon 9 rocket carrying satellites for global internet"
query3 = "300: Scientists discover a new species of deep-sea fish in the Mariana Trench"
query4 = "400: The latest iPhone model features improved camera capabilities and longer battery life"
query5 = "500: The effects of climate change on polar bear populations in the Arctic"


In [56]:
method = "BM25"
search_results_100 = basic_dictionary(query1, method, 100)
search_results_200 = basic_dictionary(query2, method, 200)
search_results_300 = basic_dictionary(query3, method, 300)
search_results_400 = basic_dictionary(query4, method, 400)
search_results_500 = basic_dictionary(query5, method, 500)

In [57]:
search_results_100

{'101': {'I Love New York': 5.02563220795661,
  'New York Harbor': 5.005606715048695,
  'New York': 4.993443713801805,
  'Lincoln Park, New Brunswick, New Jersey': 4.976605637144683,
  'New Brunswick (disambiguation)': 4.960173287918767,
  'BBC News (TV channel)': 4.954283769152033,
  'New Year': 4.953345464203204,
  'News Corp': 4.932784733001795,
  'News': 4.927235265168906,
  'Fake news': 4.923726648052557,
  'News of the World (disambiguation)': 4.9157332511009155,
  'List of bridges in the United States': 4.912079852050199,
  'Breakfast (New Zealand TV programme)': 4.895112625310964,
  'New Haven, Connecticut': 4.89305695202987,
  'Time Warner Cable News NY1': 4.879178396596843,
  'Andrea Mitchell': 4.874994900329154,
  'CBS News': 4.871520434803068,
  'Toms River, New Jersey': 4.86780076695867,
  'MetLife Stadium': 4.86664939298476,
  'New York City F.C.': 4.862982317606499,
  'Outerbridge Crossing': 4.859581605113371,
  'Brave New World (disambiguation)': 4.85795513552306,
  'Ne

# 2. Qrels

Utilisez le qrel ***qreltp*** déclaré ci-dessous

In [58]:
qreltp = {
    '101': {
        'Thomas_Müller': 1,
        'Thomas_Edison': 0,
        'Thomas_the_Apostle': 0,
    },
    '102': {
        'Mario_Gómez': 1,
        'Mario_Götze': 1,
    },
    '103': {
        'FC_Bayern_Munich': 1,
    },
    '201': {
        'Lionel_Messi': 1,
    },
    '202': {
        'Carles_Puyol': 1,
    },
    '203': {
        'FC_Bayern_Munich': 1,
    },
    '301': {
        'Skype': 1,
    },
    '302': {
        'Mac_OS': 1,
    },
    '401': {
        'Dallas_Cowboys': 1,
    },
    '402': {
        'Barack_Obama': 1,
    },
    '403': {
        'Jerry_Jones': 1,
    },
    '501': {
        'Catherine_Duchess_of_Cambridge': 1,
    },
    '502': {
        'Prince_Harry': 1,
    },
    '503': {
        'Anglicanism': 1,
    },
}

In [59]:
all_search_results = {**search_results_100, **search_results_200, **search_results_300, **search_results_400, **search_results_500}

# 3. Configurations
Générez au moins 5 configurations différents de votre système avec 100 résultats et évaluez-les. Comment expliquez-vous vos résultats ?


In [60]:
!pip install python-Levenshtein

In [92]:
# VOTRE CODE ICI
import pyterrier as pt
import Levenshtein

if not pt.started():
    pt.init()

# Fonction pour trouver le mot le plus proche dans l'index en termes de distance de Levenshtein
def correct_term(term, index_terms):
    closest_term = min(index_terms, key=lambda x: Levenshtein.distance(term, x))
    distance = Levenshtein.distance(term, closest_term)
    return closest_term if distance < 2 else term

def systeme(text, model, index):
    print(f"Evaluating configuration with model {model} and index {index}")
    res = {}
    mots = text.split()

    br = pt.BatchRetrieve(index, wmodel=model, metadata=["docno", "title", "url"])

    for mot in mots:
        mot_correct = correct_term(mot, index_terms)
        results = br.search(mot_correct).head(100)  # Limiter à 100 résultats par terme de la requête
        print(f"Results for '{mot_correct}':")
        for idx, row in results.iterrows():
            print(f"{row['docno']} - {row['title']} (URL: {row['url']})")
        res[mot] = results

    return res


# Générer au moins 5 configurations différentes
configurations = [
    {"model": "BM25", "index": indexref2},  # Exemple de configuration avec un index différent
    {"model": "PL2", "index": indexref2},
    {"model": "DPH", "index": indexref2},
    {"model": "TF_IDF", "index": indexref2},
    {"model": "BM25", "index": indexref2}
]

# Parcourir chaque configuration et évaluer ses performances
for i, config in enumerate(configurations, 1):
    print(f"Evaluating configuration {i}: {config}")
    text = "Cristiano Ronaldo is the best player"  # Texte à rechercher
    results = systeme(text, config["model"], config["index"])
    # Calculer et afficher les métriques de performance (précision, rappel, F-mesure) pour cette configuration

    # Ici, nous pourrions comparer les résultats obtenus pour voir quelle configuration fonctionne le mieux en termes de qualité des résultats.


Evaluating configuration 1: {'model': 'BM25', 'index': <org.terrier.querying.IndexRef at 0x792f293a5080 jclass=org/terrier/querying/IndexRef jself=<LocalRef obj=0x56a49d3a3970 at 0x792f293ac490>>}
Evaluating configuration with model BM25 and index <org.terrier.querying.IndexRef at 0x792f293a5080 jclass=org/terrier/querying/IndexRef jself=<LocalRef obj=0x56a49d3a3970 at 0x792f293ac490>>
Results for 'Cristiano':
772767 - Cristiano Ronaldo International Airport (URL: https://simple.wikipedia.org/wiki/Cristiano%20Ronaldo%20International%20Airport)
423193 - Christian (given name) (URL: https://simple.wikipedia.org/wiki/Christian%20%28given%20name%29)
832515 - C.D. Nacional (URL: https://simple.wikipedia.org/wiki/C.D.%20Nacional)
413907 - Camilo Ponce Enrquez (URL: https://simple.wikipedia.org/wiki/Camilo%20Ponce%20Enr%C3%ADquez)
321602 - Cristiano da Silva (URL: https://simple.wikipedia.org/wiki/Cristiano%20da%20Silva)
845434 - 2009 FIFA Pusks Award (URL: https://simple.wikipedia.org/wiki/2

VOS COMMENTAIRES ICI


# 4. Résultats
Avec les mêmes 5 configurations, générez 1000 résultats et évaluez-les. Il y a-t-il des différences dans certains métriques ? Pourquoi ?


In [86]:
import pyterrier as pt
import Levenshtein
from collections import defaultdict
import numpy as np

if not pt.started():
    pt.init()

# Liste des termes de l'index
index_terms = ["Cristiano", "Ronaldo", "best", "player"]

# Liste des mots à exclure
stop_words = ["and", "in"]  # Ajoutez les mots que vous voulez exclure ici.

# Fonction pour trouver le mot le plus proche dans l'index en termes de distance de Levenshtein
def correct_term(term, index_terms):
    closest_term = min(index_terms, key=lambda x: Levenshtein.distance(term, x))
    distance = Levenshtein.distance(term, closest_term)
    return closest_term if distance < 2 else term

def systeme(text, model, index):
    print(f"Evaluating configuration with model {model} and index {index}")
    res = {}
    mots = text.split()

    br = pt.BatchRetrieve(index, wmodel=model, metadata=["docno", "title", "url"])

    for mot in mots:
        mot_correct = correct_term(mot, index_terms)
        results = br.search(mot_correct).head(1000)  # Limiter à 1000 résultats par terme de la requête
        print(f"Results for '{mot_correct}':")
        for idx, row in results.iterrows():
            print(f"{row['docno']} - {row['title']} (URL: {row['url']})")
        res[mot] = results

    return res

def evaluate_results(results):
    # Initialisez des dictionnaires pour stocker les métriques
    precision = defaultdict(list)
    rappel = defaultdict(list)
    f_mesure = defaultdict(list)

    # Évaluez chaque résultat pour chaque configuration
    for mot, res in results.items():
        # Supposons que nous ayons une liste de documents pertinents pour chaque mot
        documents_pertinents = [doc for doc in res['docno'] if doc.startswith(mot)]
        if len(documents_pertinents) > 0:
            # Calculez les métriques pour cette configuration et ce mot
            tp = np.sum(res['docno'].isin(documents_pertinents))  # True Positives
            fp = len(res) - tp  # False Positives
            fn = len(documents_pertinents) - tp  # False Negatives

            precision[mot].append(tp / (tp + fp))
            rappel[mot].append(tp / (tp + fn))
            f_mesure[mot].append(2 * (precision[mot][-1] * rappel[mot][-1]) / (precision[mot][-1] + rappel[mot][-1]))

    return precision, rappel, f_mesure

# Configurations à évaluer
configurations = [
    {"model": "BM25", "index": indexref2},  # Exemple de configuration avec un index différent
    {"model": "PL2", "index": indexref2},
    {"model": "DPH", "index": indexref2},
    {"model": "TF_IDF", "index": indexref2}
]

# Collecter les résultats pour chaque configuration
all_results = {}
for config in configurations:
    text = "Cristiano Ronaldo is the best player"  # Texte à rechercher
    results = systeme(text, config["model"], config["index"])
    all_results[(config["model"], config["index"])] = results

# Évaluez les résultats pour chaque configuration
for config, results in all_results.items():
    precision, rappel, f_mesure = evaluate_results(results)
    print(f"\nPerformance metrics for configuration {config}:")
    for mot in index_terms:
        if mot in precision:
            print(f"Term: {mot}, Precision: {np.mean(precision[mot]):.4f}, Recall: {np.mean(rappel[mot]):.4f}, F-measure: {np.mean(f_mesure[mot]):.4f}")


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
194957 - Melky Cabrera (URL: https://simple.wikipedia.org/wiki/Melky%20Cabrera)
295317 - Randy Johnson (URL: https://simple.wikipedia.org/wiki/Randy%20Johnson)
392285 - Aaron Downey (URL: https://simple.wikipedia.org/wiki/Aaron%20Downey)
260414 - Kyle Richardson (URL: https://simple.wikipedia.org/wiki/Kyle%20Richardson)
355510 - Nahomi Kawasumi (URL: https://simple.wikipedia.org/wiki/Nahomi%20Kawasumi)
213306 - Gilbert Arenas (URL: https://simple.wikipedia.org/wiki/Gilbert%20Arenas)
341680 - Sylvain Cloutier (URL: https://simple.wikipedia.org/wiki/Sylvain%20Cloutier)
616529 - Glen Sather (URL: https://simple.wikipedia.org/wiki/Glen%20Sather)
706091 - Kyle Korver (URL: https://simple.wikipedia.org/wiki/Kyle%20Korver)
216615 - Dustin Byfuglien (URL: https://simple.wikipedia.org/wiki/Dustin%20Byfuglien)
107376 - Dennis Rodman (URL: https://simple.wikipedia.org/wiki/Dennis%20Rodman)
744223 - Emmanuel Mudiay (URL:

VOS COMMENTAIRES ICI
- Les résultats affichés fournissent un aperçu des documents retournés pour chaque terme de recherche dans différentes configurations. Cependant, sans les métriques de performance telles que la précision, le rappel et la F-mesure, il est difficile de tirer des conclusions définitives sur l'efficacité relative des configurations. Pour une évaluation complète, une analyse détaillée des métriques de performance serait nécessaire.


# 5. Analyses
Faites une comparaison entre les résultats des différents configurations. Quelles métriques ont changés ?

In [93]:
from itertools import combinations

# Convertir les configurations en chaîne de caractères pour les utiliser comme clé
config_str = lambda config: f"Model: {config['model']}, Index: {config['index']}"

# Comparer les métriques entre les configurations
for term in index_terms:
    print(f"\nComparison for term '{term}':")
    for config1, config2 in combinations(configurations, 2):
        config_str1 = config_str(config1)
        config_str2 = config_str(config2)
        print(f"\nComparison between {config_str1} and {config_str2}:")
        print(f"Precision difference: {average_precision.get(config_str1, {}).get(term, 0) - average_precision.get(config_str2, {}).get(term, 0)}")
        print(f"Recall difference: {average_recall.get(config_str1, {}).get(term, 0) - average_recall.get(config_str2, {}).get(term, 0)}")
        print(f"F-measure difference: {average_f_measure.get(config_str1, {}).get(term, 0) - average_f_measure.get(config_str2, {}).get(term, 0)}")



Comparison for term 'Cristiano':

Comparison between Model: BM25, Index: <org.terrier.querying.IndexRef at 0x792f293a5080 jclass=org/terrier/querying/IndexRef jself=<LocalRef obj=0x56a49d3a3970 at 0x792f293ac490>> and Model: PL2, Index: <org.terrier.querying.IndexRef at 0x792f293a5080 jclass=org/terrier/querying/IndexRef jself=<LocalRef obj=0x56a49d3a3970 at 0x792f293ac490>>:
Precision difference: 0
Recall difference: 0
F-measure difference: 0

Comparison between Model: BM25, Index: <org.terrier.querying.IndexRef at 0x792f293a5080 jclass=org/terrier/querying/IndexRef jself=<LocalRef obj=0x56a49d3a3970 at 0x792f293ac490>> and Model: DPH, Index: <org.terrier.querying.IndexRef at 0x792f293a5080 jclass=org/terrier/querying/IndexRef jself=<LocalRef obj=0x56a49d3a3970 at 0x792f293ac490>>:
Precision difference: 0
Recall difference: 0
F-measure difference: 0

Comparison between Model: BM25, Index: <org.terrier.querying.IndexRef at 0x792f293a5080 jclass=org/terrier/querying/IndexRef jself=<Loc

VOS COMMENTAIRES ICI

In [91]:
'''
Les résultats des comparaisons entre les différents modèles et index pour chaque terme montrent des différences nulles en ce qui concerne les métriques de précision,
 rappel et F-mesure. Cela suggère que, dans le contexte de notre ensemble de données de test, les performances des diverses configurations sont équivalentes pour
  ces termes spécifiques. Cependant, il convient de noter que ces résultats pourraient être influencés par la nature des données de test utilisées ou par
  la sensibilité limitée des métriques employées à détecter de petites variations de performance. Une analyse plus approfondie pourrait être nécessaire pour
  évaluer plus précisément les performances des différents modèles et index dans d'autres contextes ou avec des métriques plus sensibles.
'''

"\nLes résultats des comparaisons entre les différents modèles et index pour chaque terme montrent des différences nulles en ce qui concerne les métriques de précision,\n rappel et F-mesure. Cela suggère que, dans le contexte de notre ensemble de données de test, les performances des diverses configurations sont équivalentes pour\n  ces termes spécifiques. Cependant, il convient de noter que ces résultats pourraient être influencés par la nature des données de test utilisées ou par \n  la sensibilité limitée des métriques employées à détecter de petites variations de performance. Une analyse plus approfondie pourrait être nécessaire pour \n  évaluer plus précisément les performances des différents modèles et index dans d'autres contextes ou avec des métriques plus sensibles.\n"